# Processes 

Processes convert a set of resources into another. The balance of this conversion is provided by the user (see section Conversion under the Defining Resources Tutorial). A few examples are list: 


## General Setup 

Consider the processes, Wind Farm (wf) and Proton Exchange Membrane Electrolysis (pem)
These require the resource wind, power, water (h2o), hydrogen (h2), oxygen (o2)

wind and h2o can be consumed, there is a demand for hydrogen and oxygen can be released

In [1]:
from energia import Currency, Process, Resource, Model, Periods

m = Model()
m.usd = Currency()
m.q = Periods()
m.y = 4 * m.q
m.declare(Resource, ['wind', 'power', 'h2o', 'h2', 'o2'])
m.declare(Process, ['wf', 'pem'])


m.wind.consume == True
m.h2o.consume == True

m.h2.release.prep(180) >= [0.6, 0.7, 0.8, 0.3]
m.o2.release == True

--- General Resource Balance for wind in (l, y): initializing constraint, adding consume(wind, l, y)
    Completed in 0.00011754035949707031 seconds
--- General Resource Balance for h2o in (l, y): initializing constraint, adding consume(h2o, l, y)
    Completed in 8.940696716308594e-05 seconds
--- General Resource Balance for h2 in (l, q): initializing constraint, adding release(h2, l, q)
    Completed in 8.630752563476562e-05 seconds
--- Binding release in domain (h2, l, q)
    Completed in 0.00024199485778808594 seconds
--- General Resource Balance for o2 in (l, y): initializing constraint, adding release(o2, l, y)
    Completed in 7.367134094238281e-05 seconds


## With Positive Basis 

The basis is provided in the format Process(Resource | Conversion) For example, in the case of wind farm the basis is provided as power produced. Thus all the values for usd.spend need to be provided on the basis of power production 

In [2]:
m.wf(m.power) == -m.wind
m.wf.operate.prep(norm=True) <= [0.9, 0.8, 0.5, 0.7]
m.wf.capacity[m.usd.spend] == 990637 + 3354
m.wf.operate[m.usd.spend] == 49

--- Binding operate in domain (wf, l, q)
    Completed in 0.00016069412231445312 seconds
--- Mapping operate: (wf, l, q) → (wf, l, y)
    Completed in 0.000s


## With Negative Basis (or Conversion)

PEMs are typically sized based on their power consumption, thus all the values provide should be on the basis of power expended in the process

In [3]:
m.pem(-m.power) == 0.01248 * m.h2 + 0.09987 * m.o2 - 0.11235 * m.h2o
m.pem.capacity[m.usd.spend] == 1.55 * 10**6

## Locating Processes

Processes need to located explicitly 

If using multiple locations, the user can use m.location.locate(.. list of processes ..)

For single period models, it may be convenient to use m.locate(.. list of processes ..) which defaults to m.network.locate(..). Note that in a single location example, the location itself is the network

In [4]:
m.locate(m.pem, m.wf)

--- Assuming  pem capacity is unbounded in (l, y)
--- Assuming operation of pem is bound by capacity in (l, y)
--- Binding operate in domain (pem, l, y)
    Completed in 0.00010132789611816406 seconds
--- General Resource Balance for power in (l, q): initializing constraint, adding expend(power, l, q, operate, pem)
    Completed in 9.703636169433594e-05 seconds
--- Mapping operate: (pem, l, q) → (pem, l, y)
    Completed in 0.000s
--- General Resource Balance for h2 in (l, q): adding produce(h2, l, q, operate, pem)
    Completed in 9.870529174804688e-05 seconds
--- General Resource Balance for o2 in (l, y): adding produce(o2, l, y, operate, pem)
    Completed in 6.4849853515625e-05 seconds
--- General Resource Balance for h2o in (l, y): adding expend(h2o, l, y, operate, pem)
    Completed in 6.866455078125e-05 seconds
--- Assuming  wf capacity is unbounded in (l, y)
--- General Resource Balance for power in (l, q): adding produce(power, l, q, operate, wf)
    Completed in 9.27448272705

The balances arising from production are only assumed once the process has beeen located. Upon location, the model can be optimized

In [5]:
m.usd.spend.opt()
m.capacity.output()

--- Mapping spend: (usd, l, y, capacity, wf) → (usd, l, y)
    Completed in 0.000s
--- Mapping spend: (usd, l, y, operate, wf) → (usd, l, y)
    Completed in 0.000s
--- Mapping spend: (usd, l, y, capacity, pem) → (usd, l, y)
    Completed in 0.000s
--- Generating Program(m).mps
--- Creating gurobi model for Program(m)
Set parameter Username
Academic license - for non-commercial use only - expires 2026-08-01
Read MPS format model from file Program(m).mps
Reading time = 0.00 seconds
PROGRAM(M): 41 rows, 38 columns, 82 nonzeros
--- Optimizing Program(m) using gurobi
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 41 rows, 38 columns and 82 nonzeros
Model fingerprint: 0xa6fecba0
Coefficient statistics:
  Matrix range     [1e-02, 2e+06]
  Objective range  [1e+00, 1e+00]
  Bound

## Modeling Technology Choice

Let us continue with the example, but provide another option for power generation (solar PV)

In [6]:
m.solar = Resource()
m.solar.consume == True
m.pv = Process()

m.pv(m.power) == -1 * m.solar
m.pv.capacity.x <= 30000
m.pv.capacity.x >= 0
m.pv.operate.prep(norm=True) <= [0.6, 0.8, 0.9, 0.7]
m.pv.capacity[m.usd.spend] == 5670
m.pv.operate[m.usd.spend] == 90

m.locate(m.pv)

--- General Resource Balance for solar in (l, y): initializing constraint, adding consume(solar, l, y)
    Completed in 0.00011730194091796875 seconds
--- Binding capacity in domain (pv, l, y)
    Completed in 0.00010991096496582031 seconds
--- Binding capacity in domain (pv, l, y)
    Completed in 7.891654968261719e-05 seconds
--- Binding operate in domain (pv, l, q)
    Completed in 0.00034427642822265625 seconds
--- Mapping operate: (pv, l, q) → (pv, l, y)
    Completed in 0.000s
--- General Resource Balance for power in (l, q): adding produce(power, l, q, operate, pv)
    Completed in 9.703636169433594e-05 seconds
--- General Resource Balance for solar in (l, y): adding expend(solar, l, y, operate, pv)
    Completed in 6.008148193359375e-05 seconds


## Semi-continuous Bounds 

These can be set using .x for any aspect. In the below example, the maximum capacity of wind farm (if set up) is 30000
The lower bound of 500 is only applied if wind farm is set up at all. Thus the values that wind farm can take are in the semi-continuous domain $\{0\} \cup [500, 30000]$


In [7]:
m.wf.capacity.x <= 30000
m.wf.capacity.x >= 500

--- Binding capacity in domain (wf, l, y)
    Completed in 0.000164031982421875 seconds
--- Binding capacity in domain (wf, l, y)
    Completed in 0.00010991096496582031 seconds


In [8]:
m.usd.spend.opt()

--- Generating Program(m).mps
--- Creating gurobi model for Program(m)
Read MPS format model from file Program(m).mps
Reading time = 0.00 seconds
PROGRAM(M): 57 rows, 54 columns, 121 nonzeros
--- Optimizing Program(m) using gurobi
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 57 rows, 54 columns and 121 nonzeros
Model fingerprint: 0xf00bc377
Variable types: 52 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e-02, 2e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e-01, 2e+02]
Presolve removed 57 rows and 54 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 24 available processors)

S

The solution for the two models can be compared

In [9]:
m.capacity.output(compare=True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Note: binaries were only introduced in the second model

In [10]:
m.capacity.reporting.output(compare=True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>